# 数据分组运算

In [1]:
import pandas as pd
import numpy as np

In [2]:
# 分组运算后保持shape
dict_obj = {'key1' : ['a', 'b', 'a', 'b', 
                      'a', 'b', 'a', 'a'],
            'key2' : ['one', 'one', 'two', 'three',
                      'two', 'two', 'one', 'three'],
            'data1': np.random.randint(1, 10, 8),
            'data2': np.random.randint(1, 10, 8)}
df_obj = pd.DataFrame(dict_obj)
df_obj

,data1,data2,key1,key2
0,4,9,a,one
1,8,4,b,one
2,7,5,a,two
3,8,5,b,three
4,5,2,a,two
5,7,8,b,two
6,5,3,a,one
7,4,1,a,three


In [3]:
# 按key1分组后，计算data1，data2的统计信息并附加到原始表格中
k1_sum = df_obj.groupby('key1').sum().add_prefix('sum_')
k1_sum

,sum_data1,sum_data2
key1,,
a,25,20
b,23,17


In [4]:
# 方法1，使用merge
pd.merge(df_obj, k1_sum, left_on='key1', right_index=True)

,data1,data2,key1,key2,sum_data1,sum_data2
0,4,9,a,one,25,20
2,7,5,a,two,25,20
4,5,2,a,two,25,20
6,5,3,a,one,25,20
7,4,1,a,three,25,20
1,8,4,b,one,23,17
3,8,5,b,three,23,17
5,7,8,b,two,23,17


* transform方法

In [5]:
# 方法2，使用transform
k1_sum_tf = df_obj.groupby('key1').transform(np.sum).add_prefix('sum_')
df_obj[k1_sum_tf.columns] = k1_sum_tf
df_obj

,data1,data2,key1,key2,sum_data1,sum_data2,sum_key2
0,4,9,a,one,4,9,one
1,8,4,b,one,8,4,one
2,7,5,a,two,7,5,two
3,8,5,b,three,8,5,three
4,5,2,a,two,5,2,two
5,7,8,b,two,7,8,two
6,5,3,a,one,5,3,one
7,4,1,a,three,4,1,three


In [6]:
# 自定义函数传入transform
def diff_mean(s):
    """
        返回数据与均值的差值
    """
    return s - s.mean()

df_obj.groupby('key1').transform(diff_mean)

,data1,data2,sum_data1,sum_data2
0,-1.000000,5.000000,-1.000000,5.000000
1,0.333333,-1.666667,0.333333,-1.666667
2,2.000000,1.000000,2.000000,1.000000
3,0.333333,-0.666667,0.333333,-0.666667
4,0.000000,-2.000000,0.000000,-2.000000
5,-0.666667,2.333333,-0.666667,2.333333
6,0.000000,-1.000000,0.000000,-1.000000
7,-1.000000,-3.000000,-1.000000,-3.000000


In [7]:
dataset_path = './starcraft.csv'
df_data = pd.read_csv(dataset_path, usecols=['LeagueIndex', 'Age', 'HoursPerWeek', 
                                             'TotalHours', 'APM'])

* apply

In [8]:
def top_n(df, n=3, column='APM'):
    """
        返回每个分组按 column 的 top n 数据
    """
    return df.sort_values(by=column, ascending=False)[:n]

df_data.groupby('LeagueIndex').apply(top_n)

LeagueIndex   Age  HoursPerWeek  TotalHours       APM
LeagueIndex                                                            
1           2214            1  20.0          12.0       730.0  172.9530
            2246            1  27.0           8.0       250.0  141.6282
            1753            1  20.0          28.0       100.0  139.6362
2           3062            2  20.0           6.0       100.0  179.6250
            3229            2  16.0          24.0       110.0  156.7380
            1520            2  29.0           6.0       250.0  151.6470
3           1557            3  22.0           6.0       200.0  226.6554
            484             3  19.0          42.0       450.0  220.0692
            2883            3  16.0           8.0       800.0  208.9500
4           2688            4  26.0          24.0       990.0  249.0210
            1759            4  16.0           6.0        75.0  229.9122
            2637            4  23.0          24.0       650.0  227.2272
5           3277            5  18.0          16.0       950.0  372.6426
            93              5  17.0          36.0       720.0  335.4990
            202             5  37.0          14.0       800.0  327.7218
6           734             6  16.0          28.0       730.0  389.8314
            2746            6  16.0          28.0      4000.0  350.4114
            1810            6  21.0          14.0       730.0  323.2506
7           3127            7  23.0          42.0      2000.0  298.7952
            104             7  21.0          24.0      1000.0  286.4538
            1654            7  18.0          98.0       700.0  236.0316
8           3393            8   NaN           NaN         NaN  375.8664
            3373            8   NaN           NaN         NaN  364.8504
            3372            8   NaN           NaN         NaN  355.3518

In [9]:
# apply函数接收的参数会传入自定义的函数中
df_data.groupby('LeagueIndex').apply(top_n, n=2, column='Age')

LeagueIndex   Age  HoursPerWeek  TotalHours       APM
LeagueIndex                                                            
1           3146            1  40.0          12.0       150.0   38.5590
            3040            1  39.0          10.0       500.0   29.8764
2           920             2  43.0          10.0       730.0   86.0586
            2437            2  41.0           4.0       200.0   54.2166
3           1258            3  41.0          14.0       800.0   77.6472
            2972            3  40.0          10.0       500.0   60.5970
4           1696            4  44.0           6.0       500.0   89.5266
            1729            4  39.0           8.0       500.0   86.7246
5           202             5  37.0          14.0       800.0  327.7218
            2745            5  37.0          18.0      1000.0  123.4098
6           3069            6  31.0           8.0       800.0  133.1790
            2706            6  31.0           8.0       700.0   66.9918
7           2813            7  26.0          36.0      1300.0  188.5512
            1992            7  26.0          24.0      1000.0  219.6690
8           3340            8   NaN           NaN         NaN  189.7404
            3341            8   NaN           NaN         NaN  287.8128

* 禁止分组 group_keys=False

In [10]:
df_data.groupby('LeagueIndex', group_keys=False).apply(top_n)

,LeagueIndex,Age,HoursPerWeek,TotalHours,APM
2214,1,20.0,12.0,730.0,172.9530
2246,1,27.0,8.0,250.0,141.6282
1753,1,20.0,28.0,100.0,139.6362
3062,2,20.0,6.0,100.0,179.6250
3229,2,16.0,24.0,110.0,156.7380
1520,2,29.0,6.0,250.0,151.6470
1557,3,22.0,6.0,200.0,226.6554
484,3,19.0,42.0,450.0,220.0692
2883,3,16.0,8.0,800.0,208.9500
2688,4,26.0,24.0,990.0,249.0210
